In [8]:
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, BatchNormalization
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import numpy as np

In [9]:
# Define image dimensions and batch size
img_height, img_width = 128, 128
batch_size = 5

# Define directories for training and testing data
train_data_dir = "../dataset/train"
test_data_dir = "../dataset/test"

In [10]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rescale=1./255  # Normalize pixel values
)

# Data augmentation for testing images (only rescale)
test_datagen = ImageDataGenerator(rescale=1./255)


In [11]:

# Create data generators for training and testing
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=5,
    class_mode='categorical',
    shuffle=False  # No need to shuffle for evaluation
)

Found 967 images belonging to 12 classes.
Found 221 images belonging to 12 classes.


In [12]:

# Define the MobileNet base model
base_model = MobileNet(
    input_shape=(img_height, img_width, 3),  # Adjust input shape
    include_top=False,  # Exclude the fully-connected layers
    weights='imagenet'  # Pre-trained on ImageNet
)

# Freeze the base model layers
base_model.trainable = False


In [13]:

# Create the classification head
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    Dense(12, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [14]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)




Epoch 1/10
194/194 [==============================] - 33s 141ms/step - loss: 1.6520 - accuracy: 0.4550 - val_loss: 1.0772 - val_accuracy: 0.6199
Epoch 2/10
194/194 [==============================] - 13s 67ms/step - loss: 0.9718 - accuracy: 0.6991 - val_loss: 0.9606 - val_accuracy: 0.7059
Epoch 3/10
194/194 [==============================] - 13s 68ms/step - loss: 0.8129 - accuracy: 0.7653 - val_loss: 1.0388 - val_accuracy: 0.6697
Epoch 4/10
194/194 [==============================] - 13s 67ms/step - loss: 0.6274 - accuracy: 0.8159 - val_loss: 0.9897 - val_accuracy: 0.7014
Epoch 5/10
194/194 [==============================] - 13s 67ms/step - loss: 0.6038 - accuracy: 0.8180 - val_loss: 0.9004 - val_accuracy: 0.7421
Epoch 6/10
194/194 [==============================] - 13s 68ms/step - loss: 0.5539 - accuracy: 0.8511 - val_loss: 0.9658 - val_accuracy: 0.7557
Epoch 7/10
194/194 [==============================] - 13s 67ms/step - loss: 0.4731 - accuracy: 0.8749 - val_loss: 1.1055 - val_accuracy

[1.116844654083252, 0.733031690120697]

In [21]:



# # Plot training history
# plt.style.use("ggplot")
# fig, ax = plt.subplots(2, 1)
# ax[0].plot(history.history['accuracy'], color='b', label="Training Accuracy")
# ax[0].plot(history.history['val_accuracy'], color='r', label="Validation Accuracy")
# legend = ax[0].legend(loc='best', shadow=True)

# ax[1].plot(history.history['loss'], color='b', label="Training Loss")
# ax[1].plot(history.history['val_loss'], color='r', label="Validation Loss")
# legend = ax[1].legend(loc='best', shadow=True)
# plt.savefig("model/mobilenet_acc.png")

In [16]:

# Make predictions on the test data
predictions = model.predict(test_generator)


45/45 [==============================] - 2s 21ms/step


In [23]:
#saving predictions in csv file
np.savetxt("../model_saved_files/predictions/mobilenet_predictions.csv", predictions, delimiter=",")

# Save the model
# model.save("../model_saved_files/models_weights/Mobilenet.h5")